In [1]:
## running once
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from toolz.curried import *
import matplotlib.pyplot as plt
import seaborn as sns

## Loading dataset

In [3]:
from util import *

In [4]:
year, month = 2021, 6
carpark_dir = "/home_nfs/jiangyue/data/carparking"
month_dir = os.path.join(carpark_dir, f"{year}/{month:02}")
carpark_meta = load_carpark_meta(os.path.join(carpark_dir, "carpark_meta_three_weeks.geojson"))

In [5]:
#carpark_meta

#carpark_meta[carpark_meta.Agency == "HDB"]

#carpark_meta[carpark_meta.Agency == "URA"]

#carpark_meta[carpark_meta.Agency == "LTA"].head()

In [6]:
%%time
carpark_data_names = [
    "carpark_data_2021_06_01-21.json",
    #"carpark_data_2021_04_21-25.json",
    #"carpark_data_2021_04_26-30.json"
]

carpark_data = load_carpark_data_list(
    [os.path.join(carpark_dir, name) for name in carpark_data_names]
)

Loading /home_nfs/jiangyue/data/carparking/carpark_data_2021_06_01-21.json
CPU times: user 13min 2s, sys: 1min 14s, total: 14min 16s
Wall time: 14min 15s


In [7]:
carpark_data_o = carpark_data.copy()
carpark_data_o.reset_index(inplace=True)
carpark_data_o.rename(columns={"tid": "date"}, inplace=True)

carpark_data = carpark_remove_outlier(carpark_data)
carpark_data = carpark_data.fillna(method="ffill")

100%|██████████| 1918/1918 [18:22<00:00,  1.74it/s]


In [8]:
carpark_data = carpark_add_date(carpark_data)
carpark_data = carpark_remove_constant(carpark_data)
carpark_data["min"] = (carpark_data.tid % 1440 // 5).astype(str)
len(carpark_data.sid.unique())

1843

In [9]:
carpark_data['Weekday']=carpark_data['date'].dt.dayofweek.astype(str) 
df = pd.merge(carpark_data,carpark_meta[['sid','Agency']],on='sid', how='left') #label HDB and Non HDB data
df['Agency'] = df.Agency.map(lambda x: 1 if x =='HDB' else 0).astype(str)

In [12]:
df.to_csv('carpark_data_06_01-21', index = False) # Saving the data for RNN model processing

In [17]:
df.rename(columns = {"min": "mins"}, inplace=True)
df.head()

,sid,tid,AvailableLots,date,mins,Weekday,Agency
0,0,0,2797.0,2021-06-01 00:00:00,0,1,0
1,0,1,2797.0,2021-06-01 00:01:00,0,1,0
2,0,2,2797.0,2021-06-01 00:02:00,0,1,0
3,0,3,2797.0,2021-06-01 00:03:00,0,1,0
4,0,4,2797.0,2021-06-01 00:04:00,0,1,0


In [35]:
# dp = df[df.tid%5==0]
# dp.loc[:, 'tid'] = dp['tid']//5
# dp.head()

/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,sid,tid,AvailableLots,date,mins,Weekday,Agency
0,0,0,2797.0,2021-06-01 00:00:00,0,1,0
5,0,1,2799.0,2021-06-01 00:05:00,1,1,0
10,0,2,2803.0,2021-06-01 00:10:00,2,1,0
15,0,3,2805.0,2021-06-01 00:15:00,3,1,0
20,0,4,2804.0,2021-06-01 00:20:00,4,1,0


Processing raw data for XGBoost

In [42]:
t = 0
l = 20
n=30
i = 0
X = pd.DataFrame(np.random.randn(1,l+2))
Y = pd.DataFrame(np.random.randn(1,1))
for s in df.sid.unique()[0:10]:
# for s in range(100):
    print(f'Now processing carpark ID: {s}')
    df_s = df[df.sid == s]
    for t in range(len(df.tid.unique())-l-n+1):
    # for t in range(50):
        Xiter = df_s.AvailableLots[t:t+l].values.reshape(1, -1)[0]
        minsiter = df_s.mins[t:t+l+n].values[-1]
        minsiter = int(minsiter)
        Xiter = list(Xiter)
        Xiter.append(minsiter)
        dayiter = df_s.Weekday[t:t+l].values[-1]
        Xiter.append(dayiter)
        X.loc[i] = Xiter
        target = df_s.AvailableLots[t:t+l+n].values.reshape(1, -1)[0][-1]
        Y.loc[i] = target
        i += 1
print(i)

Now processing carpark ID: 0
Now processing carpark ID: 1
Now processing carpark ID: 2
Now processing carpark ID: 3
Now processing carpark ID: 4
Now processing carpark ID: 5
Now processing carpark ID: 6
Now processing carpark ID: 7
Now processing carpark ID: 8
Now processing carpark ID: 9
301910


In [44]:
X.to_csv('XGBdata', index = False) ### Save data to csv format for XGBprocess
Y.to_csv('XGBtarget', index = False)